In [12]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [13]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [ ]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VOLT_REQ', 'des_col': 'volt_req', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CUSTOMER_NAME', 'des_col': 'customer_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FATHER_HUSBAND_NAME', 'des_col': 'father_husband_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY_ID', 'des_col': 'category_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DOC_SUBMITED', 'des_col': 'doc_submited', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_2', 'des_col': 'address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METRIC', 'des_col': 'metric', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE', 'des_col': 'phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE1', 'des_col': 'phase1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AREA_NAME', 'des_col': 'area_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METRIC1', 'des_col': 'metric1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SERVICE_CONNECTION', 'des_col': 'service_connection', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REGISTRATION_NUMBER', 'des_col': 'registration_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_NAME', 'des_col': 'dtr_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LATITUDE', 'des_col': 'latitude', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BRESHNA_NO', 'des_col': 'breshna_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DATE_INSPECTION', 'des_col': 'date_inspection', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'SINGLE_THREE_MTRBOX', 'des_col': 'single_three_mtrbox', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POLE_NUM', 'des_col': 'pole_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TYPE_WIRE', 'des_col': 'type_wire', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_CODE', 'des_col': 'dtr_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INSPECTOR_NAME', 'des_col': 'inspector_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_ZONE', 'des_col': 'bill_zone', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LONGITUDE', 'des_col': 'longitude', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MOBILE_NO', 'des_col': 'mobile_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_1', 'des_col': 'address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EST_MATERIAL', 'des_col': 'est_material', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REFERENCE_NO', 'des_col': 'reference_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONTRACT_LOAD', 'des_col': 'contract_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'REGISTERED_ON', 'des_col': 'registered_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'EST_TOTAMT', 'des_col': 'est_totamt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CONNECTED_LOAD', 'des_col': 'connected_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DTR_ID', 'des_col': 'dtr_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHONE_NO', 'des_col': 'phone_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOADAPPR_ON', 'des_col': 'loadappr_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'IS_DISPATCHED', 'des_col': 'is_dispatched', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DOC_SUBMITEDM', 'des_col': 'doc_submitedm', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCON_HOURS', 'des_col': 'discon_hours', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FIELD47', 'des_col': 'field47', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCON_AMOUNT', 'des_col': 'discon_amount', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VOLTAGE', 'des_col': 'voltage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POWER_FACTOR', 'des_col': 'power_factor', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCON_PRNO', 'des_col': 'discon_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_MAXLOAD', 'des_col': 'dtr_maxload', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCONINVOICE', 'des_col': 'disconinvoice', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNT_NO', 'des_col': 'account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_4', 'des_col': 'address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PROVIENCE', 'des_col': 'provience', 'src_dtype': 'String', 'des_dtype': 'String'},
    #{'src_col': 'STATUS', 'des_col': 'status', 'src_dtype': 'integer', 'des_dtype': 'integer'},
    {'src_col': 'PAY_FLAG', 'des_col': 'pay_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NSC_TRACKING_HISTORY', 'des_col': 'nsc_tracking_history', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APP_NUM', 'des_col': 'app_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GPS_CO', 'des_col': 'gps_co', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISPATCH_NO', 'des_col': 'dispatch_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISPATCH_DT', 'des_col': 'dispatch_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_BY', 'des_col': 'approved_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EST_PRNO', 'des_col': 'est_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'WO_NUM', 'des_col': 'wo_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_NUMBER', 'des_col': 'mtr_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_VOLTAGE', 'des_col': 'dtr_voltage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESTAPPR_BY', 'des_col': 'estappr_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESTAPPR_ON', 'des_col': 'estappr_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'INVOICE', 'des_col': 'invoice', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INVOICE_PRNO', 'des_col': 'invoice_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INTIMATION_DATE', 'des_col': 'intimation_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'CURRENT_IN_AMPS', 'des_col': 'current_in_amps', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'AVAILABLESPACE_MTRBOX', 'des_col': 'availablespace_mtrbox', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APP_FEE', 'des_col': 'app_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LOAD_TYPE', 'des_col': 'load_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRID_REQUIRED', 'des_col': 'grid_required', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRID_RAISED_AT', 'des_col': 'grid_raised_at', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOAD_STATUS', 'des_col': 'load_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REPOSITORY_ID', 'des_col': 'repository_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REPOSITORY_SEC', 'des_col': 'repository_sec', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SITE_INSPECTION', 'des_col': 'site_inspection', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REJECT_REASON', 'des_col': 'reject_reason', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REJECTED_ON', 'des_col': 'rejected_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_3', 'des_col': 'address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REJECTED_BY', 'des_col': 'rejected_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEE_JSON', 'des_col': 'fee_json', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPUAPPR_LETTER', 'des_col': 'lpuappr_letter', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SECSITEINS_LETTER', 'des_col': 'secsiteins_letter', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LABAPPR_LETTER', 'des_col': 'labappr_letter', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TAZKARI_DOC', 'des_col': 'tazkari_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PROOFOFOWNERSHIP_DOC', 'des_col': 'proofofownership_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'WATER_DOC', 'des_col': 'water_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRIDEXT_MAT', 'des_col': 'gridext_mat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRID_AMOUNT', 'des_col': 'grid_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CPHONE_NO', 'des_col': 'cphone_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FAX_NO', 'des_col': 'fax_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CFAX_NO', 'des_col': 'cfax_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EMAIL', 'des_col': 'email', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'QUOTATION_TOBE', 'des_col': 'quotation_tobe', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPLIANCES', 'des_col': 'appliances', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ENG_YEAR', 'des_col': 'eng_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DARI_YEAR', 'des_col': 'dari_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CYCLE', 'des_col': 'cycle', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPLIANCE_JSON', 'des_col': 'appliance_json', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LIGHT_INDOOR', 'des_col': 'light_indoor', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LIGHT_OUTDOOR', 'des_col': 'light_outdoor', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SITE_REMARKS', 'des_col': 'site_remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SITE_REMARKSLPU', 'des_col': 'site_remarkslpu', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPUAPPROVED_ON', 'des_col': 'lpuapproved_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'METER_OWNED_BY', 'des_col': 'meter_owned_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESTIMATED_ON', 'des_col': 'estimated_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'ESTRETURN_ON', 'des_col': 'estreturn_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'ACCNUMGEN_DT', 'des_col': 'accnumgen_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'WOGEN_DT', 'des_col': 'wogen_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'MTRALLOCATED_DT', 'des_col': 'mtrallocated_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'LABAPPRV_DT', 'des_col': 'labapprv_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'MTR_INSTALLED_DT', 'des_col': 'mtr_installed_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'GRIDEXT_ON', 'des_col': 'gridext_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'NEWMTRBOXINST_DT', 'des_col': 'newmtrboxinst_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'TRNASUP_DT', 'des_col': 'trnasup_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'TRANSAPPR_ON', 'des_col': 'transappr_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'LPUSITEREJECTED_ON', 'des_col': 'lpusiterejected_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'LPUSITEAPPROV_ON', 'des_col': 'lpusiteapprov_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'SITEREJECTED_ON', 'des_col': 'siterejected_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'RELEASED_ON', 'des_col': 'released_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'REISSUED_ON', 'des_col': 'reissued_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'EXIST_LOAD', 'des_col': 'exist_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ADDITIONAL_LOAD', 'des_col': 'additional_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ESTIMATED_BY', 'des_col': 'estimated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EST_EDITED_BY', 'des_col': 'est_edited_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EST_EDITED_ON', 'des_col': 'est_edited_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'CONDITION_WIRING', 'des_col': 'condition_wiring', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_OF_FLOORS', 'des_col': 'no_of_floors', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_OF_MTR_CONNECTED', 'des_col': 'no_of_mtr_connected', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISTANCE_TRANS_MTRBOX', 'des_col': 'distance_trans_mtrbox', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISTANCE_MTRBOX_CONSSITE', 'des_col': 'distance_mtrbox_conssite', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TAX', 'des_col': 'tax', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_INSTALLED_BY', 'des_col': 'meter_installed_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INSPECTED_FROM', 'des_col': 'inspected_from', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REASON_UNHOLD', 'des_col': 'reason_unhold', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REASON_HOLD', 'des_col': 'reason_hold', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPU_REGFEE', 'des_col': 'lpu_regfee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPU_PRNO', 'des_col': 'lpu_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPU_INVOICE', 'des_col': 'lpu_invoice', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'STAGE', 'des_col': 'stage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBSTATION_ID', 'des_col': 'substation_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEEDER_ID', 'des_col': 'feeder_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_CAPACITY', 'des_col': 'dtr_capacity', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NON_REFUND_FEE', 'des_col': 'non_refund_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LPU_SITEINS_FEE', 'des_col': 'lpu_siteins_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SERV_CONN_FEE', 'des_col': 'serv_conn_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_COST', 'des_col': 'meter_cost', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'FORM_COST', 'des_col': 'form_cost', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SEAL_COST', 'des_col': 'seal_cost', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SITEINS_FEE', 'des_col': 'siteins_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DTR_TEST_FEE', 'des_col': 'dtr_test_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'ELEC_TECH_FEE', 'des_col': 'elec_tech_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CREFERENCE_NO', 'des_col': 'creference_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CEST_PRNO', 'des_col': 'cest_prno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCON_DATE', 'des_col': 'discon_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'String'},
    {'src_col': 'CPAY_FLAG', 'des_col': 'cpay_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MEGGER_TEST_FEE', 'des_col': 'megger_test_fee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OBD_VOLT_FEE', 'des_col': 'obd_volt_fee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INJECT_III_DTR_FEE', 'des_col': 'inject_iii_dtr_fee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INJECT_SYS_DTR_FEE', 'des_col': 'inject_sys_dtr_fee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_RATION_TEST_FEE', 'des_col': 'dtr_ration_test_fee', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LINE_VOLTAGE_KV', 'des_col': 'line_voltage_kv', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DISCON_TIME', 'des_col': 'discon_time', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CUSTOMER_LASTNAME', 'des_col': 'customer_lastname', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'text'},
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'}
    
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.NEWCONNECTION_REGISTER', 'public', 'newconnection_register', cols_map)
print("Total Migrated Records (newconnection_register): " + str(migrated_row_count))
